# **MODEL D: YOLOv3 + DeepSORT + Early Fused Skeleton + ST-DenseNet** 
## A unified framework for pedestrian intention prediction.
1. **YOLOv3** -> Object detector: responsible to identify and detect objects of interest in a given frame or image.

2. **DeepSORT** -> Object Tracker: DeepSORT is responsible tracking the detected objects and enhances the tracking performance by extracting additional features for person re-identification.

4. **Early Fused Skeleton** -> Skeleton mapping: Skeletons are then mapped for each tracked pedestrian.

3. **Spatio-Temporal DenseNet** -> Classifier: responsible to classify every identified and tracked pedestrian's intention by using the last 16 frames of a pedetrian.

*The codes for YOLOv3 was adapted from the GitHub repo: https://github.com/zzh8829/yolov3-tf2*

*The codes for DeepSORT was adapted from the GitHub repo: https://github.com/nwojke/deep_sort*

*The codes for Skeleton FittingTF-PoseEstimator was adapted from the GitHub repo: https://github.com/ildoonet/tf-pose-estimation*

*The codes for ST-DenseNet was adapted from the GitHub repo: https://github.com/GalDude33/DenseNetFCN-3D*



## **INSTRUCTIONS TO RUN THE MODEL ON GOOGLE COLAB**

This project was completely developed on Google Colab.



###1. Connect runtime to GPU for better/faster results.


###2. Clone the repository to Colab.

In [0]:
# run this to clone the repository Volvo-DataX
!git clone https://github.com/mjpramirez/Volvo-DataX

###3. Run this to install dependencies

In [0]:
%cd Volvo-DataX/tf-pose-estimation
! pip3 install -r requirements.txt
%cd tf_pose/pafprocess
! sudo apt install swig
!swig -python -c++ pafprocess.i && python3 setup.py build_ext --inplace

###4. Next click this link to activate the folder in your google drive: https://drive.google.com/open?id=1HxKtxBva3US2AJfohlKfjYSdhHvjt2Yc and add a shortcut of the folder to the main drive folder

And finally, run the cell to mount your google drive


In [0]:
from google.colab import drive
drive.mount('/content/drive')

###5. To run the remaining cells below, observe the comments and run them appropriately. Also running some codes may provide warnings, so please ignore them. 

In [0]:
# run this
%cd /content/Volvo-DataX

try:
  %tensorflow_version 2.x
except Exception:
  pass
import glob
 
import sys #Run this
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import (
    YoloV3, YoloV3Tiny
)
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

tf.compat.v1.disable_eager_execution()
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

flags.DEFINE_string('classes', 'data/coco.names', 'path to classes file')
flags.DEFINE_string('weights', '/content/drive/My Drive/datax_volvo_additional_files/yolov3_train_5.tf','path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_integer('num_classes', 1, 'number of classes in the model')
flags.DEFINE_string('video', 'data/JAAD_test_video_0339.mp4','path to video file or number for webcam)')
flags.DEFINE_string('output','Result_model_D.mp4', 'path to output video')
flags.DEFINE_string('output_format', 'mp4v', 'codec used in VideoWriter when saving video to file')

app._run_init(['yolov3'], app.parse_flags_with_usage)


In [0]:
%cd /content/Volvo-DataX/deep_sort
from ds_tools.generate_detections import create_box_encoder
from ds_application_util import preprocessing
from ds_deep_sort import nn_matching
from ds_deep_sort.detection import Detection
from ds_deep_sort.tracker import Tracker

%cd /content/Volvo-DataX/tf-pose-estimation
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh
from tf_pose.estimator import Human
model = TfPoseEstimator(get_graph_path('egen_jaad_1_5'), target_size=(100, 100))

%cd /content/Volvo-DataX

nms_max_overlap = 1.0
max_cosine_distance = 0.2
nn_budget = None
encoder = create_box_encoder('mars-small128.pb', batch_size=32)
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

with open('densenet_model.json', 'r') as json_file:
    json_savedModel= json_file.read()

model_j = tf.keras.models.model_from_json(json_savedModel)
model_j.load_weights('densenet_2.hdf5')

def pred_func(X_test):
  predictions = model_j.predict(X_test[0:1], verbose=0)
  Y = np.argmax(predictions[0], axis=0)
    
  return Y


In [0]:

# Run this
FLAGS.yolo_iou_threshold = 0.5
FLAGS.yolo_score_threshold = 0.5

color = (255, 0, 0) 
thickness = 2

yolo = YoloV3(classes=FLAGS.num_classes)

yolo.load_weights(FLAGS.weights).expect_partial()
logging.info('weights loaded')

class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
logging.info('classes loaded')

resize_out_ratio = 4.0

def run_model():

  print('Processing started.......')

  fps_time = 0

  try:
      vid = cv2.VideoCapture(int(FLAGS.video))
  except:
      vid = cv2.VideoCapture(FLAGS.video)

  out = None

  if FLAGS.output:
      # by default VideoCapture returns float instead of int
      width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
      height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
      fps = int(vid.get(cv2.CAP_PROP_FPS))
      codec = cv2.VideoWriter_fourcc(*FLAGS.output_format)
      out = cv2.VideoWriter(FLAGS.output, codec, fps, (width, height))

  frame = 0
  rolling_data={}
  fps_time=0

  while True:

    _, img = vid.read() # reading the image

    if img is None:
        break
        logging.warning("Empty Frame")
        time.sleep(0.1)
        continue
    frame += 1   
    currFrame = int(vid.get(cv2.CAP_PROP_POS_FRAMES))
    fps = vid.get(cv2.CAP_PROP_FPS)


    img_in = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
    img_orig = np.copy(img)
    img_in = tf.expand_dims(img_in, 0)
    img_in = transform_images(img_in, FLAGS.size)

    t1 = time.time()
    boxes, scores, classes, nums = yolo.predict(img_in, steps=1) # yolo
    t2 = time.time()
    
    # for 80 cls yolo

    boxes = boxes[:,:nums[0],:].reshape(nums[0], 4)[classes[0][:nums[0]] == 0]
    scores = scores[0][:nums[0]][classes[0][:nums[0]] == 0]
    nums = len(boxes)

    wh = np.flip(img.shape[0:2])
    bbtlwh = []
    for i in range(nums):


      x1y1 = tuple((np.array(boxes[i][0:2]) * wh).astype(np.int32))
      x1 = x1y1[0]
      y1 = x1y1[1]
      x2y2 = tuple((np.array(boxes[i][2:4]) * wh).astype(np.int32))
      bbwh = (x2y2[0]-x1y1[0], x2y2[1]-x1y1[1])
      w = bbwh[0]
      h = bbwh[1]
      bbtlwh.append([x1,y1,w,h])

    features = encoder(img, bbtlwh) # deepsort input
    detections = [Detection(box, conf, feat) for box, conf, feat in zip(bbtlwh, scores, features)] #deep sort output 

    # Update tracker.
    tracker.predict()
    tracker.update(detections)
    
    tracked_bbox = []
    ids = []

    for track in tracker.tracks:

      if not track.is_confirmed() or track.time_since_update > 1:
        continue
      tracked_bbox.append(track.to_tlwh())
      ids.append(track.track_id)


    for i in range(len(tracked_bbox)): # densenet 

      # Show tracker output
      x, y, w, h = tracked_bbox[i]
      x = int(x)  
      y = int(y) 
      w = int(w) 
      h = int(h) 

      # plot the skeletons
      try:
        cropped = img_orig[y:y + h, x:x + w]
        humans = model.inference(cropped, resize_to_default=(w > 0 and h > 0), upsample_size=resize_out_ratio)
        humans.sort(key=lambda human: human.score, reverse=True)
        skelett = TfPoseEstimator.draw_humans(cropped, humans, imgcopy=True)
        img_orig[y:y + h, x:x + w] = skelett
        img_orig2 = img_orig
      
      except:
        img_orig2 = img_orig

      # looking for previous 16 frames data for a given pedestrian:

      intent = 0 #(default, the pedestrian is not crossing)

      
      if int(ids[i]) in list(rolling_data.keys()):

        if len(rolling_data[int(ids[i])]) == 16:
          
          seq = np.stack(np.array(rolling_data[int(ids[i])]),axis=2)
          seq = np.expand_dims(seq, axis=0)
          intent = pred_func(seq) # classification output

        else:

          seq = np.stack(np.array([rolling_data[int(ids[i])][-1]] * 16),axis=2)
          seq = np.expand_dims(seq, axis=0)
          intent = pred_func(seq) # classification output
      

      # risky pedestrian identification thru box color

      if intent == 1:
        color = (0, 0, 255) # Red -> Crossing

      else:
        color = (0, 255, 0) # green -> Not crossing

      fps_time = time.time()
      img = cv2.rectangle(img_orig2, (int(x), int(y)), (int(x + w), int(y + h)), color, 2)
      img = cv2.putText(img, 'TrackID ' + str(ids[i]), (x, y - 5), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 0, 0), thickness=2)
      img = cv2.putText(img,"Frame No: %d" % (frame),(10, 10),  cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0, 0, 255), 2)

      # storing the data for last 16 frames

      try:

        if int(ids[i]) in list(rolling_data.keys()): # ID exists in dict

          if len(rolling_data[int(ids[i])]) < 16: # bboxes values for 16 frames
              
            cropped_seq = []
            cropped_img = cv2.resize(img_orig[y:h+y, x:w+x],(100,100))
            rolling_data[int(ids[i])].append(np.asarray(cropped_img)) # append the image      

          else:

            del rolling_data[int(ids[i])][0] # delete oldest frame bbox and append latest frame bbox
            cropped_seq = []
            cropped_img = cv2.resize(img_orig[y:h+y, x:w+x],(100,100))
            rolling_data[int(ids[i])].append(np.asarray(cropped_img))
              
        else:

          cropped_seq = []
          cropped_img = cv2.resize(img_orig[y:h+y, x:w+x],(100,100))
          rolling_data[int(ids[i])] = [np.asarray(cropped_img)]  

      except:
        pass

    
    if FLAGS.output:
      out.write(img)

    if cv2.waitKey(1) == ord('q'):
      break

  cv2.destroyAllWindows()
  print('\nProcessing completed.......!!!')
  print('Check video file in Volvo-DataX folder!')

  return


###6. Run this to obtain the Model-D output as a video file named **'Result_model_D.mp4'** in Volvo-DataX folder. 
After running the run_model() function expect around 15 mins for GPU and 45 mins for CPU

In [0]:
run_model()